#1. Import Necessary Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

* numpy: Used for dataset creation and numerical operations.
* matplotlib.pyplot: For visualizing the results (MSE and
$𝑅^2$
  score).
* PolynomialFeatures: Expands input features into polynomial terms, increasing model complexity.
* Ridge: Implements Ridge regression (L2 regularization) to reduce overfitting.
mean_squared_error and r2_score: Performance metrics for regression tasks.
  * MSE: Measures error magnitude.

  * $𝑅^2$ : Indicates the proportion of variance explained by the model.
* train_test_split: Splits data into training and testing sets.

#2. Generate Synthetic Dataset

In [ ]:
# Generate synthetic dataset
np.random.seed(42)
n_samples = 30
X = np.random.uniform(-1, 1, size=(n_samples, 1))
y = np.sin(2 * np.pi * X).ravel() + 0.3 * np.random.normal(size=n_samples)

* X: Generated as uniformly distributed points between -1 and 1.
* y: A sine wave with added Gaussian noise (
𝜎
= 0.3) to simulate noisy real-world data.

#3. Initialize Parameters

In [ ]:
# Regularization parameters
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4]
l2_weights = [0.0, 0.1, 0.5, 1.0, 5.0]

# Initialize results
degrees = np.arange(1, 50)
results = {(dropout_rate, l2_weight): {'train_errors': [], 'test_errors': [], 'train_r2': [], 'test_r2': []}
           for dropout_rate in dropout_rates for l2_weight in l2_weights}

* dropout_rates: Simulates feature randomization during training, ranging from 0.0 (no dropout) to 0.4 (higher dropout).
* l2_weights: L2 regularization strengths, penalizing large weights in the model.
* degrees: Polynomial degrees representing model complexity.
* results: Stores training and testing errors (MSE) and $R^2$
  scores for each combination of dropout rate and L2 regularization.


#4. Define Dropout Function

In [ ]:
# Function to simulate dropout (artificially setting weights to zero)
def apply_dropout(data, rate):
    mask = np.random.binomial(1, 1 - rate, size=data.shape)
    return data * mask

* Generates a binary mask (0 or 1) using the specified dropout rate.
* Zeroes out a proportion of features, simulating dropout behavior.

#5. Compute Results for Dropout and L2 Regularization

In [ ]:
for dropout_rate in dropout_rates:
    for l2_weight in l2_weights:
        train_errors = []
        test_errors = []
        train_r2_scores = []
        test_r2_scores = []

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

        for degree in degrees:
            poly = PolynomialFeatures(degree=degree)
            X_train_poly = apply_dropout(poly.fit_transform(X_train), dropout_rate)
            X_test_poly = apply_dropout(poly.transform(X_test), dropout_rate)

            model = Ridge(alpha=l2_weight)  # Combine L2 and dropout
            model.fit(X_train_poly, y_train)

            y_train_pred = model.predict(X_train_poly)
            y_test_pred = model.predict(X_test_poly)

            train_errors.append(mean_squared_error(y_train, y_train_pred))
            test_errors.append(mean_squared_error(y_test, y_test_pred))
            train_r2_scores.append(r2_score(y_train, y_train_pred))
            test_r2_scores.append(r2_score(y_test, y_test_pred))

        results[(dropout_rate, l2_weight)]['train_errors'] = train_errors
        results[(dropout_rate, l2_weight)]['test_errors'] = test_errors
        results[(dropout_rate, l2_weight)]['train_r2'] = train_r2_scores
        results[(dropout_rate, l2_weight)]['test_r2'] = test_r2_scores

1. Outer Loops:
* Iterate over all combinations of dropout rates and L2 regularization values.
2. Inner Loop:

Polynomial Feature Expansion:
* Expands features for each polynomial degree.
Dropout:

* Applies random feature deactivation to simulate dropout.

Ridge Regression:

* Fits the model with the given L2 regularization value.

Error and Score Calculation:
* Computes MSE and $𝑅^2$ for both training and testing datasets.
3. Result Storage:
* Errors and scores are stored for later analysis.

#6. Plot Results: Mean Squared Error (MSE)

In [ ]:
# Plot MSE results
fig, axs = plt.subplots(len(l2_weights), 1, figsize=(15, 4 * len(l2_weights)), sharex=True)
for i, l2_weight in enumerate(l2_weights):
    ax = axs[i]
    for dropout_rate in dropout_rates:
        train_errors = results[(dropout_rate, l2_weight)]['train_errors']
        test_errors = results[(dropout_rate, l2_weight)]['test_errors']

        ax.plot(degrees, test_errors, label=f"Test Loss (Dropout={dropout_rate})", marker='o')
        ax.plot(degrees, train_errors, linestyle='--', label=f"Train Loss (Dropout={dropout_rate})")

    ax.set_yscale("log")
    ax.set_title(f"L2 Regularization (Alpha={l2_weight})")
    ax.set_ylabel("MSE (Log Scale)")
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, title="Legend")
plt.xlabel("Model Complexity (Polynomial Degree)")
plt.suptitle("Effect of Combined Dropout and L2 Regularization on Double Descent (MSE)", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

* Plots training and testing MSE across model complexities for different combinations of dropout and L2 regularization.
* Logarithmic scaling highlights differences across orders of magnitude.

#7. Plot Results: $𝑅^2$ Score

In [ ]:
# Plot R2 results
fig, axs = plt.subplots(len(l2_weights), 1, figsize=(15, 4 * len(l2_weights)), sharex=True)
for i, l2_weight in enumerate(l2_weights):
    ax = axs[i]
    for dropout_rate in dropout_rates:
        train_r2 = results[(dropout_rate, l2_weight)]['train_r2']
        test_r2 = results[(dropout_rate, l2_weight)]['test_r2']

        ax.plot(degrees, test_r2, label=f"Test R² (Dropout={dropout_rate})", marker='o')
        ax.plot(degrees, train_r2, linestyle='--', label=f"Train R² (Dropout={dropout_rate})")

    ax.set_title(f"L2 Regularization (Alpha={l2_weight})")
    ax.set_ylabel("R² Score")
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, title="Legend")
plt.xlabel("Model Complexity (Polynomial Degree)")
plt.suptitle("Effect of Combined Dropout and L2 Regularization on Double Descent (R²)", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

* Visualizes $𝑅^2$
  scores for both training and testing data.
* Highlights how well the model fits the data under varying conditions.